In [1]:
import operator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn import feature_selection
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler, RobustScaler, Imputer, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from IPython.core.pylabtools import figsize 
%matplotlib inline

In [2]:
RANDOM_STATE = 42
DATA_EXTERNAL = "../data/external/"
DATA_PROCESSED = "../data/processed/"
DATA_INTERIM = "../data/interim/"

In [18]:
hum = pd.read_csv(DATA_PROCESSED + "humsavar_varq_gt.csv.gz")
# hum["MUTANT"] = hum.MUTANT.str.rstrip()
hum.drop_duplicates(inplace=True)
hum.drop_duplicates(subset="MUTANT", keep=False, inplace=True)

#Protparam
protparam = pd.read_csv(DATA_INTERIM + "protparam_features.csv.gz")
hum = hum.merge(protparam, on="MUTANT", how="left")

#Conservation
phastCons46way = pd.read_csv("../data/interim/phastCons46way.csv")
phastCons46way.rename(columns={"name": "dbSNP"}, inplace=True)
phyloP46way = pd.read_csv("../data/interim/phyloP46way.csv")
phyloP46way.rename(columns={"name": "dbSNP"}, inplace=True)
hum = hum.merge(phastCons46way, on="dbSNP", how="left")
hum = hum.merge(phyloP46way, on="dbSNP", how="left")
hum.drop("dbSNP", axis=1, inplace=True)

#VEST
hum = pd.concat([hum, hum.MUTANT.str.extract(r"(?P<WildType>\D)-(?P<Mut>\D)", expand=True)], axis=1)
AA_Features = pd.read_csv("../data/external/AA_Features_snvbox.csv")
hum = hum.merge(AA_Features, on=['WildType', 'Mut'], how='left')
hum.drop(['Mut', 'WildType'], axis=1, inplace=True)

hum.replace([np.inf, -np.inf], np.nan, inplace=True)
hum.columns = hum.columns.str.upper()

In [19]:
hum.to_csv(DATA_PROCESSED + "humsavar_gt.csv.gz", sep=",", index=False, compression="gzip")

In [17]:
hum.replace([np.inf, -np.inf], np.nan)

,MUTANT,3DID,ACTIVE_SITE,AGGREGABILITY,BFACTOR,CONSERVATION,PDB,SASA,SASA_PERCENTAGE,SWITCHBILITY,...,PAM250,BLOSUM,JM,HGMD2003,VB,Transition,COSMIC,COSMICvsSWISSPROT,HAPMAP,COSMICvsHAPMAP
0,P11362-174-V-A,False,NaN,0.000,32.2800,NaN,False,6.038,0.03,NaN,...,0.1,0,0.26,98,35,0.0043,-5.858843,-8.674093,-4.694767,-1.164076
1,Q8WZA1-504-V-I,False,NaN,NaN,67.4650,NaN,False,90.814,0.51,0.00000,...,3.2,3,1.97,87,53,0.0029,-6.352124,-9.167374,-4.447907,-1.904217
2,P46100-243-C-F,False,BINDING,NaN,0.0000,NaN,False,1.143,0.00,0.06610,...,-0.7,-2,0.69,107,3,0.0009,-7.059870,-8.916506,-6.447306,-0.612564
3,P11473-362-T-I,False,NaN,NaN,11.2900,NaN,False,0.214,0.00,0.00000,...,-0.3,-1,-0.88,151,25,0.0024,-5.076842,-7.781563,-4.397135,-0.679708
4,Q9NXN4-106-G-S,True,NaN,0.000,37.4300,0.24,False,0.000,0.00,0.04100,...,0.4,0,2.53,319,29,0.0059,-3.675480,-6.584358,-4.417135,0.741655
5,P07101-375-F-L,False,NaN,NaN,44.1225,NaN,False,4.589,0.01,0.00000,...,2.1,0,2.66,125,13,0.0042,-6.086421,-8.410852,-4.339426,-1.746995
6,P40692-68-I-N,True,NaN,0.000,44.9725,0.23,False,37.920,0.18,1.53000,...,-2.8,-3,-1.29,74,3,0.0018,-7.495188,-9.992457,-6.179042,-1.316146
7,O95497-325-A-E,False,NaN,10.032,39.7600,NaN,False,29.466,0.24,0.03170,...,-0.1,-1,0.21,54,0,0.0047,-6.715030,-9.685767,-6.447306,-0.267724
8,P13645-449-Y-C,False,NaN,NaN,200.3725,NaN,True,73.163,0.25,0.78800,...,-0.4,-2,0.25,241,5,0.0007,-4.614969,-6.624519,-4.969204,0.354235
9,P47895-89-R-C,False,NaN,0.000,15.6875,0.31,False,59.656,0.23,0.66700,...,-2.2,-3,-0.87,518,23,0.0012,-4.734028,-7.487311,-4.605536,-0.128492


In [ ]:
hum = pd.read_csv(DATA_PROCESSED + "humsavar_gt.csv.gz", sep=",")

In [ ]:
hum.columns

In [ ]:
hum.describe()

In [ ]:
hum.replace([np.inf,-np.inf], np.nan, inplace=True)

In [ ]:
model_dict = {
    'lr':{'pipe':('lr', LogisticRegression(random_state=RANDOM_STATE)),
          'params':[{'lr__C' : [.001, .01, .1, 1, 10, 100, 1000], 'lr__class_weight':[None, 'balanced']}],
          'name':'LogisticRegression'
    },
    'rf':{'pipe':('rf', RandomForestClassifier(random_state=RANDOM_STATE)),
          'params':[{'rf__max_depth':[3,5,7, 10], 'rf__n_estimators':[5, 7, 10], 'rf__max_features':[4,'sqrt',0.2, 10, 15]}],
          'name':'Random Forest'
    },
    'svc':{'pipe':('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, probability=True)),
           'params':[{'svc__C':[0.001, 0.01, 0.1, 1, 10], 'svc__gamma':[0.001, 0.01, 0.1, 1]}],
           'name': 'Support Vector Classifier'
    }
}

In [ ]:
dataset = hum.drop(['3DID', 'PDB'], 1) \
    .replace({"ACTIVE_SITE": {"BINDING": 1, np.nan: 0}}) \
    .set_index("MUTANT")

unclassified_index = dataset[dataset.TYPE == "Unclassified"].index
dataset_disease_index = dataset[dataset.TYPE == "Disease"].index
dataset_poly_index = dataset[dataset.TYPE == "Polymorphism"].index

#### 50-50% en el set de entrenamiento y 60-40% en set de evaluación respetando humsavar

In [ ]:
dataset.shape

In [ ]:
dataset.TYPE.value_counts()

In [ ]:
for i in range(20):
    train = pd.concat([
        dataset.loc[dataset_disease_index].sample(n=1300, random_state=i),
        dataset.loc[dataset_poly_index].sample(n=1300, random_state=i)
    ]).sample(frac=1)

    test = pd.concat([
        dataset.loc[dataset_disease_index.difference(train.index)].sample(n=1291, random_state=i),
        dataset.loc[dataset_poly_index.difference(train.index)].sample(n=861, random_state=i)
    ]).sample(frac=1)
    train.to_csv(DATA_PROCESSED + "train_test_sets/train_{}.tab.csv".format(i), sep="\t", index=True, index_label="MUTANT")
    test.to_csv(DATA_PROCESSED + "train_test_sets/test_{}.tab.csv".format(i), sep="\t", index=True, index_label="MUTANT")

#### Random Forest

In [ ]:
algorithm = 'rf'
model = Pipeline([('imputer', Imputer(missing_values="NaN", strategy="median")), 
                  model_dict[algorithm]['pipe']])
param_list = [model_dict[algorithm]['params']][0]
gs = GridSearchCV(model, param_list, cv=3, n_jobs=2, scoring='roc_auc', verbose=1, refit=True)

In [ ]:
rf_train_scores = []
rf_test_scores = []
for i in range(20):
    train = pd.read_csv(DATA_PROCESSED + "train_test_sets/train_{}.tab.csv".format(i), sep="\t", index_col="MUTANT")
    test = pd.read_csv(DATA_PROCESSED + "train_test_sets/test_{}.tab.csv".format(i), sep="\t", index_col="MUTANT")
    X_train = train.drop("TYPE", 1)
    y_train = train.TYPE
    X_test = test.drop("TYPE", 1)
    y_test = test.TYPE
    le = LabelEncoder().fit(y_train)
    gs.fit(X_train, le.transform(y_train))
    print("Parameters: ", gs.best_params_)
#     print("Score train ({}): {}".format(i, gs.best_score_))
    rf = gs.best_estimator_
    y_pred = rf.predict(X_test)
    test_score = roc_auc_score(le.transform(y_test), y_pred)
#     print("Score test ({}): {}".format(i, score))
    rf_test_scores.append(test_score)
    rf_train_scores.append(gs.best_score_)

In [ ]:
print("MEAN: ", np.mean(rf_train_scores))
print("STDDEV: ", np.std(rf_train_scores))

In [ ]:
print("MEAN: ", np.mean(rf_test_scores))
print("STDDEV: ", np.std(rf_test_scores))

In [ ]:
print("MEAN: ", np.mean(rf_test_scores))
print("STDDEV: ", np.std(rf_test_scores))

In [ ]:
print("MEAN: ", np.mean(test_scores))
print("STDDEV: ", np.std(test_scores))

In [ ]:
importances = [(X_train.columns[e], x) for e, x in enumerate(rf.steps[1][1].feature_importances_)]
importances.sort(key=operator.itemgetter(1), reverse=True)
figsize(20,6)
plt.title("Feature importances", fontsize=16)
plt.bar(np.arange(len(importances)), [y for x, y in importances], color="g", align="center")
plt.xticks(range(len(importances)), [x for x, y in importances], rotation='vertical', fontsize=16)
# plt.xlim([0, range(len(importances))])
plt.show()

#### Y los unclassified?

In [ ]:
pd.Series(le.inverse_transform(rf.predict(dataset.loc[unclassified_index].drop("TYPE", 1)))).value_counts()

#### Support Vector Classifier

In [ ]:
algorithm = 'svc'
model = Pipeline([('imputer', Imputer(missing_values="NaN", strategy="median")), 
                  ('scale', StandardScaler()), 
                  model_dict[algorithm]['pipe']])
param_list = [model_dict[algorithm]['params']][0]
gs = GridSearchCV(model, param_list, cv=3, n_jobs=2, scoring='roc_auc', verbose=1, refit=True)

In [ ]:
svc_train_scores = []
svc_test_scores = []
for i in range(20):
    train = pd.read_csv(DATA_PROCESSED + "train_test_sets/train_{}.tab.csv".format(i), sep="\t", index_col="MUTANT")
    test = pd.read_csv(DATA_PROCESSED + "train_test_sets/test_{}.tab.csv".format(i), sep="\t", index_col="MUTANT")
    X_train = train.drop("TYPE", 1)
    y_train = train.TYPE
    X_test = test.drop("TYPE", 1)
    y_test = test.TYPE
    le = LabelEncoder().fit(y_train)
    gs.fit(X_train, le.transform(y_train))
#     print("Score train ({}): {}".format(i, gs.best_score_))
    svc = gs.best_estimator_
    y_pred = svc.predict(X_test)
    score = roc_auc_score(le.transform(y_test), y_pred)
#     print("Score test ({}): {}".format(i, score))
    svc_train_scores.append(gs.best_score_)
    svc_test_scores.append(score)

In [ ]:
print("MEAN: ", np.mean(svc_train_scores))
print("STDDEV: ", np.std(svc_train_scores))

In [ ]:
print("MEAN: ", np.mean(svc_train_scores))
print("STDDEV: ", np.std(svc_train_scores))

In [ ]:
print("MEAN: ", np.mean(svc_train_scores))
print("STDDEV: ", np.std(svc_train_scores))

In [ ]:
print("MEAN: ", np.mean(svc_test_scores))
print("STDDEV: ", np.std(svc_test_scores))

In [ ]:
y_proba = pd.DataFrame(svc.predict_proba(X_test))[0]

In [ ]:
svc_results = pd.DataFrame({"y_pred": y_pred, "y_true": le.transform(y_test), "score": y_proba})

In [ ]:
svc_results.to_csv("../results/svc_results.csv", index=False)